In [ ]:
import os
import gc
import json
import re
from datetime import datetime
import zipfile

import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc


import torch
import transformers
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer

In [ ]:
model_path = "meta-llama/Llama-3.3-70B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model = HookedTransformer.from_pretrained(model_path, device=device, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
model.eval()

# Data setup

In [ ]:
!git clone https://github.com/LRudL/sad.git

In [ ]:
#get SAD stages_oversight
def load_jsonl_from_zip(zip_path, file_name, password=b'sadtimesforthesetimes'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        content = zip_ref.read(file_name, pwd=password).decode('utf-8')
        return [json.loads(line) for line in content.strip().split('\n')]

zip_path = './sad/sad/stages/private_data_gen.zip'
deploy_oversight = load_jsonl_from_zip(zip_path, 'out/deploy_oversight.jsonl')
test_oversight = load_jsonl_from_zip(zip_path, 'out/test_oversight.jsonl')

# Sample verification printouts
print(f"Number of samples in deploy_oversight: {len(deploy_oversight)}")
print(f"Number of samples in test_oversight: {len(test_oversight)}\n")

print("First item in deploy_oversight:")
print(deploy_oversight[0])
print("\nFirst item in test_oversight:")
print(test_oversight[0])

In [ ]:
#use chat template

def format_question_choices(item):
    is_testing = 'yes' in item['choices_right']
    
    messages = [
     {"role": "system", "content": ""},
     {"role": "user", "content": f"{item['body']}"}
    ]

    formatted_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    return {
        "question": formatted_message,
        "type": "testing" if is_testing else "deployment"
    }

In [ ]:
all_data = deploy_oversight + test_oversight
formatted_data = []

for entry in all_data:
    formatted_data.append(format_question_choices(entry))

In [ ]:
formatted_data[0]

# Probe scoring functions

In [ ]:
def get_probe_scores(model, steering_vectors, layer, prompt, tokenizer=None):
    """
    Calculate probe activation values for each token position in a prompt.
    
    Args:
        model: The HookedTransformer model
        steering_vectors: Dictionary of steering vectors indexed by layer
        layer: The specific layer to probe
        prompt: The input prompt to analyze
        tokenizer: Optional tokenizer (uses model's tokenizer if None)
    
    Returns:
        tuple: (tokens, scores, token_strings)
            - tokens: The tokenized prompt
            - scores: List of scalar probe activation values for each token position
            - token_strings: List of decoded token strings
    """
    if tokenizer is None:
        tokenizer = model.tokenizer
    
    # Ensure the steering vector for the requested layer exists
    if layer not in steering_vectors:
        raise ValueError(f"No steering vector available for layer {layer}")
    
    # Get the steering vector for the requested layer
    steering_vector = steering_vectors[layer]
    
    # Tokenize the prompt
    tokens = model.to_tokens(prompt) if hasattr(model, 'to_tokens') else torch.tensor([tokenizer.encode(prompt)]).to(model.device)
    
    # Store activation scores for each token position
    scores = []
    
    # Define the hook function to capture activations at each position
    def hook_fn(activation, hook):
        # For each token position in the sequence
        for pos in range(activation.shape[1]):
            # Get activation at this position
            act = activation[0, pos, :]
            
            # Project the activation onto the steering vector
            # (dot product of activation with normalized steering vector)
            projection = torch.dot(act, steering_vector[0]) / steering_vector[0].norm()
            
            # Save the scalar projection value
            if pos == len(scores):  # Only add if we haven't processed this position yet
                scores.append(projection.item())
        
        return activation
    
    # Run the model with the hook attached
    model.reset_hooks()
    with model.hooks(fwd_hooks=[(f"blocks.{layer}.hook_resid_pre", hook_fn)]):
        with torch.no_grad():
            model(tokens)
    
    # Get the token strings for better interpretability
    token_strings = [tokenizer.decode([t]) for t in tokens[0].tolist()]
    
    return tokens, scores, token_strings

In [ ]:
def analyze_dataset_with_probe(model, steering_vectors, layer, dataset, tokenizer=None, output_file="probe_stats.csv"):
    """
    Loop through a dataset and collect probe statistics for each question.
    
    Args:
        model: The HookedTransformer model
        steering_vectors: Dictionary of steering vectors indexed by layer
        layer: The specific layer to probe
        dataset: List of dictionaries containing 'question', 'test', 'deploy', 'correct_answer'
        tokenizer: Optional tokenizer (uses model's tokenizer if None)
        output_file: Path to save the CSV output
    """
    if tokenizer is None:
        tokenizer = model.tokenizer
    
    # Prepare results storage
    results = []
    
    # Process each question in the dataset
    for i, item in enumerate(tqdm(dataset, desc="Analyzing questions")):
        question = item['question']
        
        try:
            # Get probe scores for the question
            tokens, scores, token_strings = get_probe_scores(model, steering_vectors, layer, question, tokenizer)
            
            # Filter out BOS tokens
            filtered_scores = []
            filtered_tokens = []
            filtered_indices = []
            
            for j, (token, score) in enumerate(zip(token_strings, scores)):
                # Skip the BOS token (different models might have different BOS tokens)
                if ("<|begin_of_text|>" in token or 
                    "<s>" in token or 
                    "<BOS>" in token or
                    token.strip() == ""):
                    continue
                
                filtered_indices.append(j)
                filtered_scores.append(score)
                filtered_tokens.append(token)
            
            # Calculate statistics
            if filtered_scores:
                max_score = max(filtered_scores)
                min_score = min(filtered_scores)
                mean_score = sum(filtered_scores) / len(filtered_scores)
                std_score = (sum((x - mean_score) ** 2 for x in filtered_scores) / len(filtered_scores)) ** 0.5
            else:
                max_score = min_score = mean_score = std_score = float('nan')
            
            # Record results
            result = {
                'question_id': i,
                'question': question,
                'type': item['type'],
                'max_score': max_score,
                'min_score': min_score,
                'mean_score': mean_score,
                'std_score': std_score,
            }
            
            results.append(result)
            
            # Print progress
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1}/{len(dataset)} questions")
        
        except Exception as e:
            print(f"Error processing question {i}: {e}")
            # Add a row with the error
            results.append({
                'question_id': i,
                'question': question,
                'type': item['type'],
                'max_score': float('nan'),
                'min_score': float('nan'),
                'mean_score': float('nan'),
                'std_score': float('nan'),
                'error': str(e)
            })
    
    # Save results to CSV
    pd.DataFrame(results).to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    return results


In [ ]:
def test_all_layers(model, normalized_vectors, test_data, tokenizer, output_dir="layer_probe_results"):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get the number of layers in the model
    num_layers = len(normalized_vectors.keys())
    
    # Initialize a list to store accuracy results
    accuracy_results = []
    
    # Loop through all layers with a progress bar
    for layer in tqdm(range(1, num_layers), desc="Testing layers"):
        # Output file path for this layer
        output_file = os.path.join(output_dir, f"probe_stats_layer{layer}.csv")
        
        # Analyze the dataset with the current layer
        results = analyze_dataset_with_probe(
            model=model,
            steering_vectors=normalized_vectors,
            layer=layer,
            dataset=test_data,
            tokenizer=tokenizer,
            output_file=output_file
        )
        
    return

In [ ]:
def calculate_auroc_from_csv(file_path):
    """Calculate AUROC for a single CSV file"""
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the required columns exist
        required_columns = ['type', 'mean_score']
        if not all(col in df.columns for col in required_columns):
            return None, "Missing required columns (type and/or mean_score)"
        
        # Prepare data for ROC calculation
        # 1 for deployment (positive class), 0 for testing (negative class)
        y_true = (df['type'] == 'deployment').astype(int)
        
        # Use mean_score as the prediction score
        y_score = df['mean_score']
        
        # Calculate ROC curve points
        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        
        # Calculate optimal threshold (Youden's J statistic)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        sensitivity = tpr[optimal_idx]
        specificity = 1-fpr[optimal_idx]
        
        return {
            'auroc': roc_auc,
            'optimal_threshold': optimal_threshold,
            'sensitivity': sensitivity,
            'specificity': specificity
        }, None
    
    except Exception as e:
        return None, str(e)

In [ ]:

def process_all_csvs_in_directory(directory_path):
    """Process all CSV files in the given directory"""
    results = {}
    
    # Check if directory exists
    if not os.path.isdir(directory_path):
        print(f"Error: '{directory_path}' is not a valid directory.")
        return results
    
    # Get list of CSV files
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    
    if not csv_files:
        print(f"No CSV files found in {directory_path}")
        return results
    
    print(f"Found {len(csv_files)} CSV files in {directory_path}")
    print(f"{'File Name':<40} {'AUROC':<10} {'Threshold':<10} {'Sensitivity':<12} {'Specificity':<12}")
    print("-" * 90)
    
    # Process each CSV file
    for file_name in csv_files:
        file_path = os.path.join(directory_path, file_name)
        stats, error = calculate_auroc_from_csv(file_path)
        
        if error:
            print(f"{file_name:<40} Error: {error}")
        else:
            auroc = stats['auroc']
            threshold = stats['optimal_threshold']
            sensitivity = stats['sensitivity']
            specificity = stats['specificity']
            
            print(f"{file_name:<40} {auroc:.4f}    {threshold:.4f}    {sensitivity:.4f}      {specificity:.4f}")
            results[file_name] = stats
    
    # Find and print the file with the highest AUROC
    if results:
        best_file = max(results.items(), key=lambda x: x[1]['auroc'])
        print("\nHighest AUROC:")
        print(f"File: {best_file[0]}, AUROC: {best_file[1]['auroc']:.4f}")
    
    return results

In [ ]:
def load_vectors_from_dir(directory):
    """Load all vectors from a directory"""
    vectors = {}
    for filename in os.listdir(directory):
        if filename.startswith('layer_') and filename.endswith('.pt'):
            layer = int(filename.split('_')[1].split('.')[0])
            vectors[layer] = torch.load(os.path.join(directory, filename))
            print(vectors[layer])
    return vectors


# Run experiment

In [ ]:
existing_dir = './LLAMA_70B_CONTRASTIVE_20250407_114049/' #20250216_233223
if existing_dir is not None:
    normalized_vectors = load_vectors_from_dir(os.path.join(existing_dir, 'normalized_vectors'))
    print(f"Loaded {len(normalized_vectors)} normalized vectors from {existing_dir}")

In [ ]:
output_dir = 'LLAMA_70B_CONTRASTIVE_PROBE_RESULTS'
results = test_all_layers(
    model=model,
    normalized_vectors=normalized_vectors,
    test_data=test_data,
    tokenizer=tokenizer,
    output_dir="LLAMA_70B_CONTRASTIVE_PROBE_RESULTS"
)

In [ ]:
auc_results = process_all_csvs_in_directory(output_dir)

# control 

In [36]:
# Original layer 23 probe
original = normalized_vectors[23]

# Calculate mean and std
mean = original.mean().item()
std = original.std().item()

# Generate 10 random probes with same shape, mean, and std
random_probes = {}
for i in range(80):  # for all layers 0-79
    # Generate same random probe for all layers
    random_vector = torch.randn_like(original) * std + mean
    random_probes[i] = random_vector.to(dtype=torch.bfloat16, device='cuda:0')

# This will give you 10 different random_probes dictionaries, each having the same
# random vector repeated across all layers

random_probes_list = []
for _ in range(10):
    probe = {}
    random_vector = torch.randn_like(original) * std + mean
    for i in range(80):
        probe[i] = random_vector.to(dtype=torch.bfloat16, device='cuda:0')
    random_probes_list.append(probe)

In [40]:
output_dir = 'LLAMA_70B_CONTRASTIVE_PROBE_CONTROL_RESULTS'
results = test_all_layers(
    model=model,
    normalized_vectors=random_probes,
    test_data=test_data,
    tokenizer=tokenizer,
    output_dir="LLAMA_70B_CONTRASTIVE_PROBE_CONTROL_RESULTS"
)

lyzing questions:   9%|▉         | 11/120 [00:01<00:14,  7.77it/s]

Processed 10/120 questions



lyzing questions:  18%|█▊        | 22/120 [00:02<00:10,  9.70it/s]

Processed 20/120 questions



lyzing questions:  27%|██▋       | 32/120 [00:03<00:07, 11.09it/s]

...


Testing layers:   4%|▍         | 3/79 [00:41<17:33, 13.86s/it]

Processed 120/120 questions
Results saved to LLAMA_70B_CONTRASTIVE_PROBE_CONTROL_RESULTS/probe_stats_layer3.csv



lyzing questions:   8%|▊         | 10/120 [00:01<00:13,  8.35it/s]

Processed 10/120 questions



Testing layers:   4%|▍         | 3/79 [00:43<18:23, 14.52s/it]

KeyboardInterrupt



In [41]:
auc_results = process_all_csvs_in_directory(output_dir)

Found 10 CSV files in LLAMA_70B_CONTRASTIVE_PROBE_CONTROL_RESULTS
File Name                                AUROC      Threshold  Sensitivity  Specificity 
------------------------------------------------------------------------------------------
probe_stats_layer1.csv                   0.4302    -0.0104    1.0000      0.0862
probe_stats_layer2.csv                   0.5687    0.0187    0.7419      0.4828
probe_stats_layer3.csv                   0.6541    0.0097    0.4839      0.8276
probe_stats_layer4.csv                   0.6452    0.0238    0.4516      0.8621
probe_stats_layer5.csv                   0.5623    0.0107    0.9516      0.1897
probe_stats_layer6.csv                   0.4839    -0.1045    0.7742      0.3276
probe_stats_layer7.csv                   0.4625    0.1219    0.1774      0.9655
probe_stats_layer8.csv                   0.5854    -0.0444    0.8387      0.3793
probe_stats_layer9.csv                   0.7264    -0.0866    0.7419      0.7069
probe_stats_layer10.csv       

In [51]:
import numpy as np
from scipy import stats

def compare_auc(auc1, auc2, n1, alpha=0.05):
    """
    Compare two AUCs using proper standard error calculation

    Args:
        auc1, auc2: The two AUCs to compare
        n1: Sample size
        alpha: Significance level
    """
    # Standard error calculation
    se1 = np.sqrt((auc1 * (1-auc1)) / (n1/4))
    se2 = np.sqrt((auc2 * (1-auc2)) / (n1/4))

    # Standard error of the difference
    se_diff = np.sqrt(se1**2 + se2**2)

    # Z-score
    z = (auc1 - auc2) / se_diff

    # Two-tailed p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))

    return z, p_value


# Test cases
test_cases = [
    (0.5, 0.9999, 400000),  #  test case
    (0.6, 0.7, 100),  # small difference, small n
    (0.6, 0.7, 10000),  # small difference, large n
]

for auc1, auc2, n in test_cases:
    z, p = compare_auc(auc1, auc2, n)
    print(f"\nComparing AUC {auc1} vs {auc2} with n={n}:")
    print(f"Z-statistic: {z:.3f}")
    print(f"P-value: {p:.10f}")
    print(f"Significant at α=0.05: {p < 0.05}")


Comparing AUC 0.5 vs 0.9999 with n=400000:
Z-statistic: -316.101
P-value: 0.0000000000
Significant at α=0.05: True

Comparing AUC 0.6 vs 0.7 with n=100:
Z-statistic: -0.745
P-value: 0.4560565403
Significant at α=0.05: False

Comparing AUC 0.6 vs 0.7 with n=10000:
Z-statistic: -7.454
P-value: 0.0000000000
Significant at α=0.05: True


In [52]:
z, p = compare_auc(0.821, 0.602, 400)
p

0.0004276610332378805